In [1]:
import numpy as np
import pandas as pd
import re
pd.set_option('display.max_rows', 200)
from env import api_token

## Note for the team:
### This notebook is a mess. I would not try to run it if I were you.

In [2]:
import lyricsgenius as genius
def get_lyrics(title, artist):
    api = genius.Genius(api_token, verbose=False)
    song = api.search_song(title, artist)
    lyrics = song.lyrics.replace('\n', ' ')
    return lyrics

In [3]:
df = pd.read_csv('songs.csv').drop(columns=['Unnamed: 0'])

In [4]:
df = df[df.lyrics.isna()]
df = df.drop(columns=['status'])

### title_2 = remove special characters and numbers
### artist_2 = remove special characters

In [5]:
df['title_2'] = df.title.str.replace('.', ' ')
df['title_2'] = df.title_2.str.replace(r'[^a-zA-Z\s\:]', '', regex=True)
df['artist_2'] = df.artist.str.replace(r'[^a-zA-Z0-9\s\:]', '', regex=True)
df.index = range(len(df))

/var/folders/zb/3lg9b5xn3831bhkh23bd5bs00000gn/T/ipykernel_35682/4312783.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['title_2'] = df.title.str.replace('.', ' ')


In [6]:
df

,title,artist,date,lyrics,title_2,artist_2
0,"""B"" Girls",Young And Restless,1990-05-05,NaN,B Girls,Young And Restless
1,"""Cherry Cherry"" from Hot August Night",Neil Diamond,1973-03-17,NaN,Cherry Cherry from Hot August Night,Neil Diamond
2,"""Having A Party"" Medley",The Ovations (Featuring Louis Williams),1973-10-13,NaN,Having A Party Medley,The Ovations Featuring Louis Williams
3,"""Yep!""",Duane Eddy His Twangy Guitar And The Rebels,1959-03-30,NaN,Yep,Duane Eddy His Twangy Guitar And The Rebels
4,'98 Thug Paradise,"Tragedy, Capone, Infinite",1998-09-19,NaN,Thug Paradise,Tragedy Capone Infinite
...,...,...,...,...,...,...
1912,pride.is.the.devil,J. Cole & Lil Baby,2021-05-29,NaN,pride is the devil,J Cole Lil Baby
1913,punchin'.the.clock,J. Cole,2021-05-29,NaN,punchin the clock,J Cole
1914,the.climb.back,J. Cole,2021-05-29,NaN,the climb back,J Cole
1915,transparentsoul,Willow Featuring Travis Barker,2021-06-26,NaN,transparentsoul,Willow Featuring Travis Barker


In [ ]:
# title_2 and artist_2

for i in range(len(df)):
    
    title = df.iloc[[i]].title_2.values[0]
    artist = df.iloc[[i]].artist_2.values[0]
    
    try:
        df.loc[i, 'lyrics'] = get_lyrics(title, artist)
        df.loc[i, 'error'] = False
    except: 
        df.loc[i, 'error'] = True
    
    if i % 100 == 0:
        df.to_csv('songs_2.csv')
    
    print(f'{round((i / len(df) * 100), 2)}%\t complete', end='\r')
    
    df.to_csv('songs_2.csv')

In [ ]:
df.error.value_counts()

In [ ]:
df = pd.read_csv('songs_2.csv').drop(columns=['Unnamed: 0'])
df = df[df.lyrics.isna()]
df.index = range(len(df))

### title_3 = everything inside quotations

In [ ]:
df['title_3'] = df.title.str.extract(r'\"(.*)\"')

In [ ]:
# title_3 and artist_2

for i in range(len(df)):   
    
    # if title_3 is not null
    if ~df.iloc[[i]].title_3.isna().values[0]:
    
        title = df.iloc[[i]].title_3.values[0]
        artist = df.iloc[[i]].artist_2.values[0]

        try:
            df.loc[i, 'lyrics'] = get_lyrics(title, artist)
            df.loc[i, 'error'] = False
        except: 
            df.loc[i, 'error'] = True

        if i % 100 == 0:
            df.to_csv('songs_3.csv')
    
    print(f'{round((i / len(df) * 100), 2)}%\t complete', end='\r')
    
    df.to_csv('songs_3.csv')

In [ ]:
df.error.value_counts()

In [ ]:
df = pd.read_csv('songs_3.csv').drop(columns=['Unnamed: 0'])

In [ ]:
df = df[df.lyrics.isna()]
df.index = range(len(df.index))
df.shape

### artist_3 = remove everything after "Feat"

In [ ]:
df['artist_3'] = df.artist_2.str.extract(r'(.+)Feat')

In [ ]:
# title_2, artist_3

for i in range(len(df)):   
    
    # if artist_3 is not null
    if ~df.iloc[[i]].artist_3.isna().values[0]:
    
        title = df.iloc[[i]].title_2.values[0]
        artist = df.iloc[[i]].artist_3.values[0]

        try:
            df.loc[i, 'lyrics'] = get_lyrics(title, artist)
            df.loc[i, 'error'] = False
        except: 
            df.loc[i, 'error'] = True

        if i % 100 == 0:
            df.to_csv('songs_4.csv')
    
    print(f'{round((i / len(df) * 100), 2)}%\t complete', end='\r')
    
    df.to_csv('songs_4.csv')

In [ ]:
df = pd.read_csv('songs_4.csv').drop(columns=['Unnamed: 0'])

In [ ]:
df = df[~df.lyrics.isna()]
df.index = range(len(df))

In [ ]:
df.shape

### artist_4 = remove everything after "With"

In [ ]:
df['artist_4'] = df.artist_2.str.extract(r'(.+)With')

In [ ]:
# title_2, artist_4

for i in range(len(df)):   
    
    # if artist_4 is not null
    if ~df.iloc[[i]].artist_4.isna().values[0]:
    
        title = df.iloc[[i]].title_2.values[0]
        artist = df.iloc[[i]].artist_4.values[0]

        try:
            df.loc[i, 'lyrics'] = get_lyrics(title, artist)
            df.loc[i, 'error'] = False
        except: 
            df.loc[i, 'error'] = True

        if i % 100 == 0:
            df.to_csv('songs_5.csv', index=False)
    
    print(f'{round((i / len(df) * 100), 2)}%\t complete', end='\r')
    
    df.to_csv('songs_5.csv', index=False)

In [ ]:
df.error.value_counts()

In [ ]:
df

In [ ]:
songs = pd.read_csv('songs.csv').drop(columns=['Unnamed: 0'])
songs = songs[~songs.lyrics.isna()]
songs.head()

In [ ]:
songs_2 = pd.read_csv('songs_2.csv').drop(columns=['Unnamed: 0'])
songs_2 = songs_2[~songs_2.lyrics.isna()]
songs_2.head()

In [ ]:
songs_3 = pd.read_csv('songs_3.csv').drop(columns=['Unnamed: 0'])
songs_3 = songs_3[~songs_3.lyrics.isna()]
songs_3.head()

In [ ]:
songs_4 = pd.read_csv('songs_4.csv').drop(columns=['Unnamed: 0'])
songs_4 = songs_4[~songs_4.lyrics.isna()]
songs_4.head()

In [ ]:
songs_5 = pd.read_csv('songs_5.csv')
songs_5 = songs_5[~songs_5.lyrics.isna()]
songs_5.head()

In [ ]:
df = pd.concat([songs, songs_2, songs_3, songs_4, songs_5])

In [ ]:
df.shape

In [ ]:
df.index = range(len(df))

In [ ]:
from nltk.corpus import stopwords
def remove_stopwords(text, stopword_list=stopwords.words('english')):
    words = text.split()
    filtered_words = [word for word in words if word not in stopword_list]
    return ' '.join(filtered_words)

In [ ]:
# pull the title portion out of the lyrics string
df['test_title'] = df.lyrics.str.extract(r'(.+)Lyrics')
# clean it up
df['test_title'] = df.test_title.str.lower()
df['test_title'] = df.test_title.str.replace(r'[^a-zA-Z\s\:]', '', regex=True)
df['test_title'] = df.test_title.str.strip()
df['test_title'] = df.test_title.apply(remove_stopwords)
# clean up the title to match the test title
df['clean_title'] = df.title.str.replace(r'[^a-zA-Z\s\:]', '', regex=True)
df['clean_title'] = df.clean_title.str.strip()
df['clean_title'] = df.clean_title.str.lower()
df['clean_title'] = df.clean_title.apply(remove_stopwords)


In [ ]:
df.head(1)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(12,8))
sns.histplot(df[df.clean_title != df.test_title].date)

In [ ]:
plt.figure(figsize=(12,8))
sns.histplot(df.date)

In [ ]:
songs_0526 = df[df.clean_title == df.test_title]
songs_0526 = songs_0526[['title', 'artist', 'date', 'lyrics']]
#songs_0526.to_csv('songs_0526.csv')